rules of the game : https://cdn.1j1ju.com/medias/85/5e/7c-pickomino-rulebook.pdf

subject : https://team.inria.fr/polaris/files/2023/10/project_2023.pdf

#PART 1: MDP for 1 player

In [ ]:
import random as rd
import itertools
import math

\go faire un truc déguelasse avec dico et sets et ensuite on optimise si besoin

on considère dans toute la suite que 6 est synonyme de 'W' (worm)

a state is : a sorted list of dices rolled + a set of faces you cannot choose + the sum of dices already picked

In [ ]:
def compute_prob(t):
  #compute the probability of rolling the dices in t
  n = len(t)
  res = math.factorial(n)
  for i in range(1,7):
    res /= math.factorial(t.count(i))
  return int(res)/6**n


In [ ]:
#test iterator and compute_prob

it = itertools.combinations_with_replacement([1,2,3,4,5,6],3)

s = 0
for x in it:
  print(x,compute_prob(x))
  s+=compute_prob(x)

print(s)

In [ ]:
class MDP():

  def __init__(self,c,r):
    """
    Initialization
    """
    self.c = c #value when lost
    self.r = r #reward vector
    self.value = {} #value of each state
    self.opti = {} #optimal move at each state, None if terminal state

  def explore(self,dices,used,score):
    """
    Dynamic programming starting with state (dices,tuple(sorted(used)),score)
    """
    state = dices,tuple(sorted(used)),score

    #memoization
    if state in self.value:
      return self.value[state]

    nb_dices = len(dices)
    rolled = set(dices)
    choices = rolled.difference(used)

    #check for terminal state

    if nb_dices==0:
      #we gathered all the dices
      self.opti[state] = None
      if (score<21):
        #the score is too low
        self.value[state] = c
        return c
      elif 6 not in used:
        #did not get a worm
        self.value[state] = c
        return c
      else:
        self.value[state] = r[score-21]
        return r[score-21]

    if not(choices):
      #we lost because all dices are already used
      self.value[state] = c
      self.opti[state] = None
      return c


    max_value = -100000000000#don't put zero because the maximum value can be negative
    max_idx = None

    for f in choices:
      #choose face f
      count = dices.count(f)
      new_used = used.copy()
      new_used.add(f)
      if (f<6):
        new_score = score + count*f
      else:
        new_score = score + 5*count

      s = 0
      it = itertools.combinations_with_replacement([1, 2, 3, 4, 5, 6], nb_dices-count)
      for new_dices in it:
          new_value = self.explore(new_dices, new_used, new_score)
          prob = compute_prob(new_dices)
          s += new_value * prob

      if s > max_value:
          max_value = s
          max_idx = f

    if (score<21):
        #the score is too low
      stop_value = c
    elif 6 not in used:
      stop_value = c
    else:
      stop_value = r[score-21]

    if stop_value > max_value:
      self.value[state] = stop_value
      self.opti[state] = "STOP"
    else:
      self.value[state] = max_value
      self.opti[state] = max_idx
    return self.value[state]



  def explore_all(self):
    """
    Dynamic programming starting from all possibles rolls of 8 dices
    """
    it = itertools.combinations_with_replacement([1,2,3,4,5,6], 8)
    for dices in it:
        print(dices)
        self.explore(dices, set(), 0)

  def compute_value_total(self):
    """
    Computes the value of the game, needs to be called after explore_all
    """
    it = itertools.combinations_with_replacement([1,2,3,4,5,6], 8)
    p_total = 0
    for dices in it:
        prob = compute_prob(dices)
        p_total += prob * self.value[(dices, (), 0)]
    print(p_total)
    return p_total


In [ ]:
c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()

mdp.explore(dices,used,0)

state = dices, tuple(sorted(used)), score

print(state, mdp.value[state], mdp.opti[state])


((1, 3, 3, 3, 3, 4, 4, 5, 6), (), 0) 1.232827668101253 4


In [ ]:
c = -3 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()

mdp.explore(dices,used,0)

state = dices, tuple(sorted(used)), score

print(state, mdp.value[state], mdp.opti[state])


((1, 3, 3, 3, 3, 4, 4, 5, 6), (), 0) 0.2562805612151674 4


In [ ]:
c = 0 #value if you lose the turn
r = [0,0,0,1] + [0]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

(1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, 1, 1, 1, 1, 2)
(1, 1, 1, 1, 1, 1, 1, 3)
(1, 1, 1, 1, 1, 1, 1, 4)
(1, 1, 1, 1, 1, 1, 1, 5)
(1, 1, 1, 1, 1, 1, 1, 6)
(1, 1, 1, 1, 1, 1, 2, 2)
(1, 1, 1, 1, 1, 1, 2, 3)
(1, 1, 1, 1, 1, 1, 2, 4)
(1, 1, 1, 1, 1, 1, 2, 5)
(1, 1, 1, 1, 1, 1, 2, 6)
(1, 1, 1, 1, 1, 1, 3, 3)
(1, 1, 1, 1, 1, 1, 3, 4)
(1, 1, 1, 1, 1, 1, 3, 5)
(1, 1, 1, 1, 1, 1, 3, 6)
(1, 1, 1, 1, 1, 1, 4, 4)
(1, 1, 1, 1, 1, 1, 4, 5)
(1, 1, 1, 1, 1, 1, 4, 6)
(1, 1, 1, 1, 1, 1, 5, 5)
(1, 1, 1, 1, 1, 1, 5, 6)
(1, 1, 1, 1, 1, 1, 6, 6)
(1, 1, 1, 1, 1, 2, 2, 2)
(1, 1, 1, 1, 1, 2, 2, 3)
(1, 1, 1, 1, 1, 2, 2, 4)
(1, 1, 1, 1, 1, 2, 2, 5)
(1, 1, 1, 1, 1, 2, 2, 6)
(1, 1, 1, 1, 1, 2, 3, 3)
(1, 1, 1, 1, 1, 2, 3, 4)
(1, 1, 1, 1, 1, 2, 3, 5)
(1, 1, 1, 1, 1, 2, 3, 6)
(1, 1, 1, 1, 1, 2, 4, 4)
(1, 1, 1, 1, 1, 2, 4, 5)
(1, 1, 1, 1, 1, 2, 4, 6)
(1, 1, 1, 1, 1, 2, 5, 5)
(1, 1, 1, 1, 1, 2, 5, 6)
(1, 1, 1, 1, 1, 2, 6, 6)
(1, 1, 1, 1, 1, 3, 3, 3)
(1, 1, 1, 1, 1, 3, 3, 4)
(1, 1, 1, 1, 1, 3, 3, 5)
(1, 1, 1, 1, 1, 3, 3, 6)


In [ ]:
c = 0 #value if you lose the turn
r = [0,0,0,0,0,0,1] + [1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

(1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, 1, 1, 1, 1, 2)
(1, 1, 1, 1, 1, 1, 1, 3)
(1, 1, 1, 1, 1, 1, 1, 4)
(1, 1, 1, 1, 1, 1, 1, 5)
(1, 1, 1, 1, 1, 1, 1, 6)
(1, 1, 1, 1, 1, 1, 2, 2)
(1, 1, 1, 1, 1, 1, 2, 3)
(1, 1, 1, 1, 1, 1, 2, 4)
(1, 1, 1, 1, 1, 1, 2, 5)
(1, 1, 1, 1, 1, 1, 2, 6)
(1, 1, 1, 1, 1, 1, 3, 3)
(1, 1, 1, 1, 1, 1, 3, 4)
(1, 1, 1, 1, 1, 1, 3, 5)
(1, 1, 1, 1, 1, 1, 3, 6)
(1, 1, 1, 1, 1, 1, 4, 4)
(1, 1, 1, 1, 1, 1, 4, 5)
(1, 1, 1, 1, 1, 1, 4, 6)
(1, 1, 1, 1, 1, 1, 5, 5)
(1, 1, 1, 1, 1, 1, 5, 6)
(1, 1, 1, 1, 1, 1, 6, 6)
(1, 1, 1, 1, 1, 2, 2, 2)
(1, 1, 1, 1, 1, 2, 2, 3)
(1, 1, 1, 1, 1, 2, 2, 4)
(1, 1, 1, 1, 1, 2, 2, 5)
(1, 1, 1, 1, 1, 2, 2, 6)
(1, 1, 1, 1, 1, 2, 3, 3)
(1, 1, 1, 1, 1, 2, 3, 4)
(1, 1, 1, 1, 1, 2, 3, 5)
(1, 1, 1, 1, 1, 2, 3, 6)
(1, 1, 1, 1, 1, 2, 4, 4)
(1, 1, 1, 1, 1, 2, 4, 5)
(1, 1, 1, 1, 1, 2, 4, 6)
(1, 1, 1, 1, 1, 2, 5, 5)
(1, 1, 1, 1, 1, 2, 5, 6)
(1, 1, 1, 1, 1, 2, 6, 6)
(1, 1, 1, 1, 1, 3, 3, 3)
(1, 1, 1, 1, 1, 3, 3, 4)
(1, 1, 1, 1, 1, 3, 3, 5)
(1, 1, 1, 1, 1, 3, 3, 6)
